## Содержание:
* [1. Отправка списка страниц на индексирование в Яндекс Вебмастер](#1)
* [-- Настройки](#2)
* [-- Загрузка URL для индексирования](#3)
* [-- Отправка на индексирование](#4)




* [2. Получение списка проиндексированных страниц](#5)
* [-- Настройки](#6)
* [-- Получение списка страниц](#7)
* [-- Сопоставление полученных url из Вебмастера с актуальными адресами на сайте](#8)


# Отправка списка страниц на индексирование в Яндекс Вебмастер<a class="anchor" id="1"></a>

In [1]:
import json
import requests

import pandas as pd

## Настройки<a class="anchor" id="2"></a>

In [2]:
TOKEN = 'y0_AgABCAAZqca9AAaElQAAAADKLs4Yq_tVuPHCZOKIQLy3rAYljzWccks' #укажите свой токен

**user id**

In [3]:
USERID_URL = 'https://api.webmaster.yandex.net/v4/user'

**отправка на переобход**

In [ ]:
RECRAWL_URL_TEMPLATE = 'https://api.webmaster.yandex.net/v4/user/{}/hosts/{}/recrawl/queue'

Документация <a href='https://yandex.ru/dev/webmaster/doc/dg/reference/host-recrawl-post.html'>https://yandex.ru/dev/webmaster/doc/dg/reference/host-recrawl-post.html</a>


**параметры проекта**

In [4]:
host = 'https:megaposm.com:443' #укажите host из Яндекс Вебмастера для своего проекта

In [5]:
project_name = 'megaposm' #как называется файл без расширения

## Загрузка URL для индексирования<a class="anchor" id="3"></a>

Файл в формате xlsx с названием поля "urls":

In [6]:
table_with_urls_for_recrawl = pd.read_excel(f'{project_name}.xlsx')

## Отправка на индексирование <a class="anchor" id="4"></a>

### Авторизация

In [7]:
def get_auth_headers():
    return {'Authorization': f'OAuth {TOKEN}'}

### Получение user_id Яндекс Вебмастера

In [8]:
def get_user_id():
    r = requests.get(USERID_URL, headers=get_auth_headers())
    user_id = json.loads(r.text)['user_id']
    return user_id

### Отправка на индексирование

Функция принимает датафрейм, <br>
перебирает каждый url из списка и отправляет его на переобход, пока будет получен ответ НЕ 202,<br>
возвращает список отправленных url.

In [14]:
def send_urls_to_yandex_for_recrawl(data, user_id, host):

    sent_urls_for_recrawl = []

    for url in data['urls']:
        
        yandex_webmaster_recrawl_url = RECRAWL_URL_TEMPLATE.format(user_id, host)

        url_for_recrawl = {'url': f'{url}'}

        r = requests.post(yandex_webmaster_recrawl_url,
                          headers = get_auth_headers(),
                          json = url_for_recrawl
                         )

        if r.status_code != 202:
            break

        sent_urls_for_recrawl.append(url)

    sent_urls_list_len = len(sent_urls_for_recrawl)

    print(json.loads(r.text))
    print('На переобход отправлено:', sent_urls_list_len)
    
    return sent_urls_for_recrawl

### Удаление из списка и экспорт

Функция принимает датафрейм и список отправленных url,<br>
Из загруженной таблицы удаляются url, которые были отправлены. Это нужно для того, чтобы в следующий раз, когда обновится квота, эти же страницы не оправлялись на переобход.

**Экспорт**

In [10]:
def export_data_to_excel(data):
    project_report = pd.ExcelWriter(f'{project_name}.xlsx', engine='xlsxwriter')
    data.to_excel(project_report, index=False)
    project_report.save()

**Удаление и экспорт**

In [11]:
def delete_sent_urls_and_export_new_table(data, sent_urls_for_recrawl):
    data = data[~data['urls'].isin(sent_urls_for_recrawl)]
    export_data_to_excel(data)

## Главная функция

In [15]:
def main_send_pages_to_yandex_for_recrawl(data):
    user_id = get_user_id()
    
    sent_urls_for_recrawl = send_urls_to_yandex_for_recrawl(data, user_id, host)
    
    delete_sent_urls_and_export_new_table(data, sent_urls_for_recrawl)

## Запуск

In [16]:
main_send_pages_to_yandex_for_recrawl(table_with_urls_for_recrawl)

{'daily_quota': 90, 'exceeded_until': '2022-11-12T00:00:00.000+03:00', 'error_message': 'Daily quota exceeded', 'error_code': 'QUOTA_EXCEEDED'}
На переобход отправлено: 90


# Получение списка проиндексированных страниц<a class="anchor" id="5"></a>

In [83]:
import json
import requests
import os
from datetime import datetime


from bs4 import BeautifulSoup
import pandas as pd

## Настройки<a class="anchor" id="6"></a>

In [24]:
TOKEN = 'y0_AgABCAAZqca9AAaElQAAAADKLs4Yq_tVuPHCZOKIQLy3rAYljzWccks' #укажите свой токен

**user id**

In [25]:
USERID_URL = 'https://api.webmaster.yandex.net/v4/user'

**получение статистики по проекту**

In [26]:
HOST_STAT_URL_TEMPLATE = 'https://api.webmaster.yandex.net/v4/user/{}/hosts/{}/summary'

Документация <a href='https://yandex.ru/dev/webmaster/doc/dg/reference/host-id-summary.html'>https://yandex.ru/dev/webmaster/doc/dg/reference/host-id-summary.html</a>


**получение списка url в поисковой базе**

In [27]:
INSEARCH_URL_TEMPLATE = 'https://api.webmaster.yandex.net/v4/user/{}/hosts/{}/search-urls/in-search/samples'

Документация <a href='https://yandex.ru/dev/webmaster/doc/dg/reference/hosts-indexing-insearch-samples.html#hosts-indexing-insearch-samples'>https://yandex.ru/dev/webmaster/doc/dg/reference/hosts-indexing-insearch-samples.html#hosts-indexing-insearch-samples</a>

<em>**Для получения списка страниц необходимо указать количество страниц, получаемое за одну итерацию, максимально можно указать 100**</em>

In [28]:
YANDEX_PER_REQUEST_PAGE_LIMIT = 100

**параметры проекта**

In [58]:
host = 'https:megaposm.com:443' #укажите host из Яндекс Вебмастера для своего проекта

In [30]:
project_name = 'megaposm' #как называется файл без расширения

In [80]:
sitemap_path = 'https://megaposm.com/index.php?route=extension/feed/google_sitemap'

**директория для визульного отчета**

In [54]:
YANDEX_VISUAL_REPORTS_DIR = 'results/visual_reports/yandex'

**директория для файлов со списком url на переобход**

In [55]:
YANDEX_RECRAWL_BASES_DIR = 'results/recrawl_bases/yandex'

## Получение списка страниц<a class="anchor" id="7"></a>

### Авторизация

In [17]:
def get_auth_headers():
    return {'Authorization': f'OAuth {TOKEN}'}

### Получение user_id Яндекс Вебмастера

In [18]:
def get_user_id():
    r = requests.get(USERID_URL, headers=get_auth_headers())
    user_id = json.loads(r.text)['user_id']
    return user_id

### Получение URL из поисковой базы

Функция принимает user id и хост,<br>

получает число проиндексированных страниц - indexed_pages_quantity,<br>

указаны параметры offset - смещение списка, т.е. с какой строки в списке будут получены url и limit - сколько страниц будет получено за один шаг - ранее был указан этот параметр в YANDEX_PER_REQUEST_PAGE_LIMIT,<br>

запускается цикл while, который работает пока offser меньше числа проиндексированных страниц - indexed_pages_quantity,<br>

запускается генератор множеств (set), в котороый добавляются полученные url,<br>

полученный set с адресами объединяется с общим множеством indexed_pages,<br>

к offset прибавляется число полученных url YANDEX_PER_REQUEST_PAGE_LIMIT,<br>

функция возвращает множество indexed_pages.


In [78]:
def yandex_get_indexed_pages(user_id, host):
    host_stat_url = HOST_STAT_URL_TEMPLATE.format(user_id, host)
    insearch_url = INSEARCH_URL_TEMPLATE.format(user_id, host)

    r = requests.get(host_stat_url, headers=get_auth_headers())
    indexed_pages_quantity = json.loads(r.text)['searchable_pages_count']
    
    print(f'В поисковой базе содержится {indexed_pages_quantity} страниц',)

    indexed_pages = set()
    params = {'offset': 0, 'limit': YANDEX_PER_REQUEST_PAGE_LIMIT}
    while params['offset'] < indexed_pages_quantity:
        r = requests.get(insearch_url, headers=get_auth_headers(), params=params)
        current_urls = {element['url'] for element in json.loads(r.text)['samples']}
        indexed_pages = indexed_pages.union(current_urls)
        params['offset'] += YANDEX_PER_REQUEST_PAGE_LIMIT
        
    return indexed_pages

Отмечу, что Яндекс Вебмастер предоставляет только 50 000 строк с url, которые хранятся в его базе. Если на сайте больше страниц, то дальнейшее сопоставление даст некорректные результаты.

## Сопоставление полученных url из Вебмастера с актуальными адресами на сайте<a class="anchor" id="8"></a>

Функция возвращает set для удобного сопоставления с url, которые у вас на сайте.<br>

Актуальный список url вы можете получить при парсинге краулером, например Comparser. Затем загрузите таблицу, например с названием поля 'urls', где будут перечислены адреса:<br>

In [62]:
parsed_urls = pd.read_excel(f'{project_name}.xlsx')

Конвертация в set:

In [36]:
site_pages = set(parsed_urls['urls'])

Далее для сопоставления я буду использовать адреса из sitemap.

### Парсинг Sitemap

Можно указать user agent, чтобы парсер не получил ответ 403 из-за настроек сервера или CDN, например CloudFlare:

In [63]:
user_agent = {'User-agent': 'Mozilla/5.0'}

**Одна карта**

Функция принимает адрес карты сайта,<br>

парсит,<br>

возвращает set с адресами из карты сайта.

In [81]:
def parse_sitemap_one_map(sitemap_path):
    r = requests.get(sitemap_path, headers=user_agent)
    soup = BeautifulSoup(r.text, 'xml')
    url_set = {url.text for url in soup.find_all('loc')}
    
    print('Количество страниц в Sitemap:', len(url_set))
    
    return url_set

**Мультакарта**

Функция принимает адрес карты сайта,<br>

парсит, получает список вложенных карт,<br>

парсит по очереди вложенные карты,<br>

возвращает set со всеми адресами из мультикарты.

In [82]:
def parse_sitemap_many_maps(sitemap_path):

    r = requests.get(sitemap_path, headers=user_agent)
    soup = BeautifulSoup(r.text, 'xml')
    maps = [sitemap.text for sitemap in soup.find_all('loc')]
    
    url_set = set()
    for sitemap in maps:
        r = requests.get(sitemap, headers=user_agent)
        soup = BeautifulSoup(r.text, 'xml')
        current_urls = {url.text for url in soup.find_all('loc')}
        url_set = url_set.union(current_urls)
    
    print('Количество страниц в Sitemap:', len(url_set))

    return url_set

## Сопоставление адресов из базы Яндекса с адресами из Sitemap

### Сопоставление

Функция принимает set с адресами из базы Яндекса и set с адресами из карты сайта,<br>

в pages_in_index_but_not_in_sitemap записываются адреса, которые есть в поисковой базе, но отсутствуют в карте сайта,<br>

в pages_is_sitemap_but_not_in_index записываются адреса, которые есть в карте сайта, но отсутствуют в поисковой базе,<br>

оба отчета записываются в соседние колонки датафрейма для дальнейшего визуального сопоставления,<br>

функция возвращает датафрейм.

In [66]:
def comapre_indexed_pages_with_pages_in_sitemap(indexed_pages, pages_in_sitemap):
    pages_in_index_but_not_in_sitemap = indexed_pages - pages_in_sitemap
    pages_is_sitemap_but_not_in_index = pages_in_sitemap - indexed_pages

    pages_in_index_but_not_in_sitemap_df = pd.DataFrame(
        pages_in_index_but_not_in_sitemap,
        columns=['pages_in_index_but_not_in_sitemap']
    )
    pages_is_sitemap_but_not_in_index_df = pd.DataFrame(
        pages_is_sitemap_but_not_in_index,
        columns=['pages_is_sitemap_but_not_in_index']
    )
    df = pd.concat([pages_in_index_but_not_in_sitemap_df, pages_is_sitemap_but_not_in_index_df], axis=1)
    return df

### Создание визуального отчета в таблице

**Функция с настройками для экспорта**

In [67]:
def export_report_data_to_excel(data, path, filename):
    os.makedirs(path, exist_ok=True)
    project_report = pd.ExcelWriter(os.path.join(path, filename), engine='xlsxwriter')
    data.to_excel(project_report, index=False)
    project_report.save()

**Экспорт визуального отчета**

Функция принимает название проекта, путь, по которому будет лежать отчет, set с url в индексе, set url из карты сайта,<br>

затем запускается функция сопоставления и запись датафрейма в data,<br>

экспортируется файл, название которого состоит из названия проекта и даты отчета.

In [69]:
def make_visual_report_for_yandex(project_name, path, indexed_pages, pages_in_sitemap):
    data = comapre_indexed_pages_with_pages_in_sitemap(indexed_pages, pages_in_sitemap)
    current_date = datetime.now().strftime('%d_%m_%Y')
    filename = f'{project_name}_{current_date}.xlsx'
    export_report_data_to_excel(data, path, filename)

**Экспорт визуального отчета для сайтов с мультикартой**

Отдельная функция для сайтов с мультикартами. В такой карте карт может быть более 65 тысяч адресов, формат xlsx не поддерживает такое количество на одном листе, нужно записать результаты в csv.

In [70]:
def make_visual_report_for_yandex_for_many_maps(project_name, path, indexed_pages, pages_in_sitemap):
    data = comapre_indexed_pages_with_pages_in_sitemap(indexed_pages, pages_in_sitemap)
    current_date = datetime.now().strftime('%d_%m_%Y')
    filename = f'{project_name}_{current_date}.csv'
    data.to_csv(f'{path}/{filename}', index=False)

### Создание списка на переобход

Похожая функция как для сопоставления, которая принимает set с адресами из базы Яндекса и set с адресами из карты сайта,<br>

в pages_in_index_but_not_in_sitemap записываются адреса, которые есть в поисковой базе, но отсутствуют в карте сайта,<br>

в pages_is_sitemap_but_not_in_index записываются адреса, которые есть в карте сайта, но отсутствуют в поисковой базе,<br>

оба отчета записываются в одну колонку для отправки на переобход,<br>

функция возвращает датафрейм.

In [71]:
def get_recrawl_base_for_yandex(indexed_pages, pages_in_sitemap):
    pages_in_index_but_not_in_sitemap = indexed_pages - pages_in_sitemap
    pages_is_sitemap_but_not_in_index = pages_in_sitemap - indexed_pages
    all_pages_to_recrawl = list(pages_in_index_but_not_in_sitemap.union(pages_is_sitemap_but_not_in_index))
    data = pd.DataFrame(all_pages_to_recrawl, columns=['urls'])
    return data

**Экспорт таблицы со списком на переобход**

In [72]:
def make_recrawl_base_for_yandex(project_name, path, indexed_pages, pages_in_sitemap):
    data = get_recrawl_base_for_yandex(indexed_pages, pages_in_sitemap)
    filename = f'{project_name}.xlsx'
    export_report_data_to_excel(data, path, filename)

**Экспорт таблицы со списком на переобход для сайтов с мультикартой**

In [73]:
def make_recrawl_base_for_yandex_many_maps(project_name, path, indexed_pages, pages_in_sitemap):
    data = get_recrawl_base_for_yandex(indexed_pages, pages_in_sitemap)
    filename = f'{project_name}.csv'
    data.to_csv(f'{path}/{filename}', index=False)

## Главная функция

In [85]:
def main_yandex_index_sitemap_compare():
    user_id = get_user_id()
    
    print(project_name)

    indexed_pages = yandex_get_indexed_pages(user_id, host)
    pages_in_sitemap = parse_sitemap_one_map(sitemap_path)
    
    print('Подготовка визуального отчета')
    make_visual_report_for_yandex(project_name, YANDEX_VISUAL_REPORTS_DIR, indexed_pages, pages_in_sitemap)
    
    print('Подготовка таблицы со списком url на переобход')
    make_recrawl_base_for_yandex(project_name, YANDEX_RECRAWL_BASES_DIR, indexed_pages, pages_in_sitemap)

## Запуск

In [86]:
main_yandex_index_sitemap_compare()

megaposm
В поисковой базе содержится 905 страниц
Количество страниц в Sitemap: 881
Подготовка визуального отчета
Подготовка таблицы со списком url на переобход
